# Final home assignment. Iphone or Not

**Zhanna Azizova**

**Task**
 - You should to create model to detect iphone (all versions) on a picture. 
 - If picture contains two or more iphones you should return only one probability for all picture. 
 - Picture is a typical for internet shop. Solution should contains all ML stages (you can skip collect data stage) and pretrained model in git or link to another data storage. 
 - Also you should to provide example running inference.

*Interface to detection:*
- python predict.py 
- --model path_to_model 
- --input path_to_input_data 
- --output path_to_results
- input data - folder with images 
- output data - csv file with two columns: image_name, iphone_probability

In [1]:
# import libraries
import os
import zipfile
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
import warnings

from keras.optimizers import Adam
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint

from tensorflow.contrib import lite

from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix

warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Data and model architecture preparation

The best way to quickly build a good Deep neural network is to train alredy pre-trained model. One of the most powerful models is ResNet18, so thanks to Alexander's advice I have chosen this one.

In [25]:
from classification_models.resnet import ResNet18

model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=True)

model.layers.pop() #remove the last network's layer
last_layer = model.layers[-1].output
new_layer = Dense(2, activation="softmax")(last_layer)
model = Model(model.input, new_layer) #add a new layer

In [4]:
# compile model with adam optimizer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# unzip the folder with train and test data (images)

local_zip = 'data_phones.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('train_test_data')
zip_ref.close()

In [11]:
# specify path for train and test folders

train_data = 'train_test_data/data_phones/train'
test_data = 'train_test_data/data_phones/test'

In [12]:
# rescale train and test image data 

train_datagen = ImageDataGenerator(rescale = 1./255.)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [13]:
# define a batch size
b_size = 32

# flow train and test images using image data generator

train_gen = train_datagen.flow_from_directory(train_data,
                                              batch_size = b_size,
                                              class_mode = 'categorical', 
                                              target_size = (224, 224))     

test_gen = test_datagen.flow_from_directory(test_data,
                                            batch_size  = b_size,
                                            class_mode  = 'categorical', 
                                            target_size = (224, 224))

Found 59360 images belonging to 2 classes.
Found 4084 images belonging to 2 classes.


**For validation set there were equally-sized samples (2024 images for each class: iphone / other_brand). However, for train set I have put approximately 35000 images of iphone class and 25000 images of other brands' phones. So the train sample was actually impalances.**

## Training and squeezing the model 

In [13]:
# set checkpoint to not loose the training results

model_path="best_model.hdf5"
callback = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=False)

In [ ]:
# fit the model on batches (of 32 images) with real-time data augmentation

model.fit_generator(train_gen,
                    validation_data = test_gen,
                    steps_per_epoch = train_gen.samples / b_size,
                    epochs = 80,
                    validation_steps = test_gen.samples / b_size,
                    verbose = 1,
                    callbacks=[callback])

Epoch 1/80
1855/1855 [==============================] - 978s 527ms/step - loss: 0.5894 - acc: 0.6751 - val_loss: 0.9877 - val_acc: 0.5600

Epoch 00001: val_acc improved from -inf to 0.55999, saving model to best_model.hdf5
Epoch 2/80
1855/1855 [==============================] - 965s 520ms/step - loss: 0.4573 - acc: 0.7764 - val_loss: 0.4191 - val_acc: 0.8073

Epoch 00002: val_acc improved from 0.55999 to 0.80730, saving model to best_model.hdf5
Epoch 3/80
1855/1855 [==============================] - 959s 517ms/step - loss: 0.4034 - acc: 0.8050 - val_loss: 0.4260 - val_acc: 0.8073

Epoch 00003: val_acc did not improve from 0.80730
Epoch 4/80
1855/1855 [==============================] - 966s 521ms/step - loss: 0.3731 - acc: 0.8234 - val_loss: 0.4211 - val_acc: 0.8217

Epoch 00004: val_acc improved from 0.80730 to 0.82174, saving model to best_model.hdf5
Epoch 5/80
1855/1855 [==============================] - 963s 519ms/step - loss: 0.3500 - acc: 0.8352 - val_loss: 0.2881 - val_acc: 0.869


Epoch 00050: val_acc did not improve from 0.94221
Epoch 51/80
1855/1855 [==============================] - 981s 529ms/step - loss: 0.1521 - acc: 0.9339 - val_loss: 0.1552 - val_acc: 0.9383

Epoch 00051: val_acc did not improve from 0.94221
Epoch 52/80
1855/1855 [==============================] - 982s 529ms/step - loss: 0.1507 - acc: 0.9345 - val_loss: 0.1568 - val_acc: 0.9434

Epoch 00052: val_acc improved from 0.94221 to 0.94344, saving model to best_model.hdf5
Epoch 53/80
1855/1855 [==============================] - 980s 529ms/step - loss: 0.1476 - acc: 0.9361 - val_loss: 0.1506 - val_acc: 0.9425

Epoch 00053: val_acc did not improve from 0.94344
Epoch 54/80
1855/1855 [==============================] - 981s 529ms/step - loss: 0.1466 - acc: 0.9363 - val_loss: 0.1535 - val_acc: 0.9420

Epoch 00054: val_acc did not improve from 0.94344
Epoch 55/80
1855/1855 [==============================] - 988s 533ms/step - loss: 0.1458 - acc: 0.9365 - val_loss: 0.1623 - val_acc: 0.9332

Epoch 00055:

** As we can see the best model reached 95% accuracy on the validation data**

In [4]:
tflite_converter = lite.TFLiteConverter.from_keras_model_file('best_model.hdf5')
squeezed_model = tflite_converter.convert()
open("phone_classification_model.tflite", "wb").write(squeezed_model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 100 variables.
INFO:tensorflow:Converted 100 variables to const ops.


46791104

## Model evaluation 

In [34]:
# function for loading and rescaling images

def prepare_image(path):
    
    loaded_image = image.load_img(path, target_size=(224, 224))
    processed_image = image.img_to_array(loaded_image)
    processed_image = np.expand_dims(processed_image, axis=0)
    processed_image /= 255.
    
    return processed_image

In [104]:
def run_predicting(model_path, input_folder):

    model = tf.lite.Interpreter(model_path=model_path)
    model.allocate_tensors()
    input_tensors = model.get_input_details()
    output_tensors = model.get_output_details()

    probas = []
    preds = []
    classes = []
    
    for subdir, dirs, files in os.walk(input_folder):
        for i, file in enumerate(files):
            input_data = prepare_image(os.path.join(subdir, file))
            model.set_tensor(input_tensors[0]['index'], input_data)
            model.invoke()
            probability = model.get_tensor(output_tensors[0]['index'])
            
            probas.append(probability[0][0])
            
            classes.append(os.path.basename(subdir))
            
            if probability[0][0] > 0.5:
                preds.append(1)
            else:
                preds.append(0)
            
    return classes,probas, preds

In [105]:
cl, prob, pred = run_predicting('phone_classification_model.tflite', test_data)

In [110]:
labs = {'iphone':1,'other_brands':0}
y_true = [labs[x] if x in labs else 'none' for x in cl]

In [145]:
print("Average precision score: {}".format(average_precision_score(y_true, prob)))
print("Precision score: {}".format(precision_score(y_true, pred, average = 'macro')))
print("Recall score: {}".format(recall_score(y_true, pred, average = 'macro')))
print("F1 score: {}".format(f1_score(y_true, pred, average='macro')))
print("ROC AUC score: {}".format(roc_auc_score(y_true, prob)))

Average precision score: 0.9919196965339646
Precision score: 0.9513440375074876
Recall score: 0.951028403525955
F1 score: 0.9510198403227605
ROC AUC score: 0.9911210802337974


In [130]:
confusion_matrix(y_true, pred)

array([[1915,  127],
       [  73, 1969]])

## Running inference

**After model is saved, I have created a predict.py file with arguments required and launch the comand below in the terminal**

python3 predict.py  --model "phone_classification_model.tflite" --input "train_test_data/data_phones/test" --output "predictions.csv"

**After script running was finished, I got the csv file "predictions.csv" with probabilities for images in the test data to contain an iphone picture. Each probability is assigned to image file name**